# Function Calling

## Basics Concepts

> [!IMPORTANT]
> The text of this section is based on this [.txt's blog post by Will Kurt](https://blog.dottxt.co/oss-v-gpt4.html).

In June 2023, [OpenAI announced](https://openai.com/index/function-calling-and-other-api-updates/) that they were going to add support for “function calling capabilities” in their API. **Function calling** has since become a common interface to the LLM where the user provides a list of one or more “functions” and accompanying arguments for that function. The model can then choose to call a function and provide the necessary arguments.

Here is an example from the evaluation set showing how a function calling works. Here is a function called  `get_current_weather`:

```python
"function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
```

 As we can see the model now has access to a function it can “call” (really it just returns the function name and argument values, nothing ‘happens’ during function calling).

The function, `get_current_weather` has two required arguments: `location` and `format`.

When prompted with a question such as:

> What is the weather in Philadelphia?

The model can “call” the function by replying with a response like this: 

```python
[get_current_weather(location='Philadelphia, PA', format='farenheit')]
```

The main benefit of function calling is that we have a reliable method for having an LLM communicate with other systems and provide a formatted response. It’s easy to image this code being transformed into an actual function call in a language like Python or turned into a request to an API endpoint. 

## Basic Example

> [!IMPORTANT]
> Code and text based on this [Openai Cookbook example](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).

In [1]:
from dotenv import load_dotenv
load_dotenv();
import json
from openai import OpenAI
from formatting import pprint_messages
GPT_MODEL = "gpt-4o"
client = OpenAI()

First let's define a way for making calls to the Chat Completions API.

In [2]:
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

### How to generate function arguments

Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [4]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append({"role": "assistant", "content": assistant_message.content})
pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What's the weather like today?                                                                  │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please specify the location for which you'd like to know the weather?                 │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [5]:
messages.append({"role": "user", "content": "I'm in Santiago, Chile."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    messages.append({"role": "assistant", "content": f"{assistant_message.tool_calls[0].function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What's the weather like today?                                                                  │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please specify the location for which you'd like to know the weather?                 │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  I'm in Santiago, Chile.                                                                         │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location":"Santiago, Chile","format":"celsius"}',                         │
│  name='get_current_weather')                                                                     │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

By prompting it differently, we can get it to target the other function we've told it about.

In [6]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is the weather going to be like in Santiago, Chile over the next x days?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append({"role": "assistant", "content": assistant_message.content})
pprint_messages(messages)


╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the weather going to be like in Santiago, Chile over the next x days?                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please specify the number of days for the weather forecast in Santiago, Chile?        │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.

In [7]:
messages.append({"role": "user", "content": "3 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    messages.append({"role": "assistant", "content": f"{assistant_message.tool_calls[0].function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the weather going to be like in Santiago, Chile over the next x days?                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please specify the number of days for the weather forecast in Santiago, Chile?        │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  3 days                                                                                          │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location":"Santiago, Chile","format":"celsius","num_days":3}',            │
│  name='get_n_day_weather_forecast')                                                              │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

### Forcing the use of specific functions

We can force the model to use a specific function, for example `get_n_day_weather_forecast` by using the `tool_choice` argument. By doing so, we force the model to make assumptions about how to use it.

In [8]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Buenos Aires, Argentina."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    messages.append({"role": "assistant", "content": f"{assistant_message.tool_calls[0].function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Give me a weather report for Buenos Aires, Argentina.                                           │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location":"Buenos Aires, Argentina","format":"celsius","num_days":5}',    │
│  name='get_n_day_weather_forecast')                                                              │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

### Parallel Function Calling

Newer models can call multiple functions in one turn.

In [9]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is the weather going to be like in Santiago and Buenos Aires over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model=GPT_MODEL
)

assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    for tool_call in assistant_message.tool_calls:
        messages.append({"role": "assistant", "content": f"{tool_call.function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the weather going to be like in Santiago and Buenos Aires over the next 4 days          │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location": "Santiago, Chile", "format": "celsius", "num_days": 4}',       │
│  name='get_n_day_weather_forecast')                                                              │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location": "Buenos Aires, Argentina", "format": "celsius", "num_days":    │
│  4}', name='get_n_day_weather_forecast')                                                         │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

## Function Calling Lifecycle

| ![fc-diagram](function-calling-diagram-resized.png) | 
|:--:| 
| *The lifecycle of a function call. Image from [OpenAI](https://platform.openai.com/docs/guides/function-calling/lifecycle).* |


When you use the OpenAI API with function calling, the model never actually executes functions itself, instead in step 3 the model simply generates parameters that can be used to call the function, which the code can then choose how to handle, likely by calling the indicated function. The application is always in full control.

### How to call functions with model generated arguments

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/).


| ![db-diagram](sqlite-sample-db.jpg) | 
|:--:| 
| *Chinook sample database tables. Image from [Sqlite Tutorial](https://www.sqlitetutorial.net/sqlite-sample-database/).* |


First let's define some helpful utility functions to extract data from a SQLite database.

In [55]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

Opened database successfully


In [56]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


Now can use these utility functions to extract a representation of the database schema.

In [57]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [58]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

Now let's implement the function that will actually excute queries against the database.

In [61]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

Steps to invoke a function call using Chat Completions API:

- **Step 1:** Prompt the model with content that may result in model selecting a tool to use. The description of the tools such as a function names and signature is defined in the 'Tools' list and passed to the model in API call. If selected, the function name and parameters are included in the response.
- **Step 2:** Check programmatically if model wanted to call a function. If true, proceed to step 3.
- **Step 3:** Extract the function name and parameters from response, call the function with parameters. Append the result to messages.
- **Step 4:** Invoke the chat completions API with the message list to get the response.

In [87]:
# Step #1: Prompt with content that may result in function call. In this case the model can identify the information requested by the user is potentially available in the database schema passed to the model in Tools description. 
messages = [{
    "role":"user", 
    "content": "What is the name of the album with the most tracks?"
}]

response = client.chat.completions.create(
    model='gpt-4o', 
    messages=messages, 
    tools= tools, 
    tool_choice="auto"
)

# Append the message to messages list
assistant_message = response.choices[0].message 
if len(assistant_message.tool_calls) > 0:
    for tool_call in assistant_message.tool_calls:
        messages.append({"role": "function", "content": f"{tool_call.function}"})

pprint_messages(messages)

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the name of the album with the most tracks?                                             │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"query":"SELECT albums.Title, COUNT(tracks.TrackId) as TrackCount FROM     │
│  albums JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY albums.AlbumId ORDER BY          │
│  TrackCount DESC LIMIT 1;"}', name='ask_database')                                               │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [88]:
# Step 2: determine if the response from the model includes a tool call.   
# drop the last message from the list
messages.pop()
tool_calls = assistant_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = json.loads(tool_calls[0].function.arguments)['query']

    messages.append({
            "role": "assistant", 
            "tool_calls": tool_calls, 
            "content": f"{tool_calls[0].function}"
        })

    # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_query_string)
        
        messages.append({
            "role": "tool", 
            "tool_call_id": tool_call_id, 
            "name": tool_function_name, 
            "content": results
        })
        
        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        assistant_message = model_response_with_function_call.choices[0].message
        messages.append({"role": "assistant", "content": assistant_message.content})
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    messages.append({"role": "assistant", "content": assistant_message.content})

In [91]:
pprint_messages(messages)

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the name of the album with the most tracks?                                             │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"query":"SELECT albums.Title, COUNT(tracks.TrackId) as TrackCount FROM     │
│  albums JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY albums.AlbumId ORDER BY          │
│  TrackCount DESC LIMIT 1;"}', name='ask_database')                                               │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Tool ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  [('Greatest Hits', 57)]                                                                         │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  The album with the most tracks is "Greatest Hits," which contains 57 tracks.                    │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

## Resources

- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [OpenAI Cookbook: How to call functions with chat models](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models)
- [.txt: Beating GPT-4 at function calling with Open Source](https://blog.dottxt.co/oss-v-gpt4.html)